**This notebook is used to generate the random instances of the Wishart Planted Ensemble in Julia Language.**

Code by Estelle Inack.

In [ ]:
using LinearAlgebra
using Random
using DelimitedFiles

N_list = [32]  # number of spins 
# α_list = [0.25,0.5, 0.75, 1.0, 1.25, 1.5]
# α_list = [0.125]
α_list = [3/32]

num_realizations = 25

rng_seeds = [MersenneTwister(i*100 + 10000)  for i in 1:num_realizations]

25-element Array{MersenneTwister,1}:
 MersenneTwister(UInt32[0x00002774], Random.DSFMT.DSFMT_state(Int32[-11926851, 1073606668, 1650665704, 1073471581, -1879960341, 1073439488, -1027724573, 1073196559, -149516174, 1073422011  …  733802864, 1072724939, 1573146613, 1073318988, 1514839799, -1450448959, -1632994230, -1942974185, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000

In [ ]:
"""
A function that generates a single Wishart planted realization
"""
function generate_wpe(N, α, rng, identity_matrix, A)     
   
    thres = 1e-12
    M = Int(α * N)
    
    Z = rand(rng,-1:2:1, N, M)
    W = (A * Z)/sqrt(N*(N-1))
    
    Jriggle = - (W * W')/N 
    J = Jriggle - diag(Jriggle)' .* identity_matrix
        
    for i in 1:N
        for m in 1:N
            if(abs(J[i,m]) < thres) 
                J[i,m] = 0.0
            end
        end
    end
    
    return round.(J, digits=10)
    
end

"""
This function that generates many single Wishart planted realizations
"""
function wpe_examples(n, α, identity_matrix, A)
    couplers = Array{Float64}(undef, n*num_realizations, n)
    for num in 1:num_realizations
        
        println("Example: ", num, " Size: ", n, " α: ", α)
        coupler = generate_wpe(n, α, rng_seeds[num], identity_matrix, A)
                
        couplers[1+n*(num-1):n*num, :] = coupler
    end
    
    return couplers
end


"""
This function generares realizations for differents N and α 
"""
function generate_examples()
    
    dict_couplers = Dict()
    for n in N_list
        
        identity_matrix = Matrix{Int}(I, n, n)
    
        A = fill(-1, n, n)
        [A[i,i] = n-1 for i in 1:n]
                
        for α in α_list
            couplers = wpe_examples(n, α, identity_matrix, A)
            dict_couplers["size_$(n)_alpha_$(α)"] = couplers
        end
    end
    
    return dict_couplers
end

generate_examples

In [ ]:
couplings = generate_examples()

for n in N_list        
    
    for α in α_list
    
        for num in 1:num_realizations
            filename = open("data/wpe_size$(n)_alpha$(α)_realization$(num).txt", "w")
            writedlm(filename, couplings["size_$(n)_alpha_$(α)"][1+n*(num-1):n*num, :], '\t')
            close(filename)
        end
        
    end
end

Example: 1 Size: 32 α: 0.09375
Example: 2 Size: 32 α: 0.09375
Example: 3 Size: 32 α: 0.09375
Example: 4 Size: 32 α: 0.09375
Example: 5 Size: 32 α: 0.09375
Example: 6 Size: 32 α: 0.09375
Example: 7 Size: 32 α: 0.09375
Example: 8 Size: 32 α: 0.09375
Example: 9 Size: 32 α: 0.09375
Example: 10 Size: 32 α: 0.09375
Example: 11 Size: 32 α: 0.09375
Example: 12 Size: 32 α: 0.09375
Example: 13 Size: 32 α: 0.09375
Example: 14 Size: 32 α: 0.09375
Example: 15 Size: 32 α: 0.09375
Example: 16 Size: 32 α: 0.09375
Example: 17 Size: 32 α: 0.09375
Example: 18 Size: 32 α: 0.09375
Example: 19 Size: 32 α: 0.09375
Example: 20 Size: 32 α: 0.09375
Example: 21 Size: 32 α: 0.09375
Example: 22 Size: 32 α: 0.09375
Example: 23 Size: 32 α: 0.09375
Example: 24 Size: 32 α: 0.09375
Example: 25 Size: 32 α: 0.09375
